# This notebook ...

In [1]:
import pathlib
import sys
import yaml

import pandas as pd
import torch
import torch.optim as optim
import mlflow
import optuna
import joblib

## Read config

In [2]:
with open(pathlib.Path('.').absolute().parent.parent / "config.yml", "r") as file:
    config = yaml.safe_load(file)

## Import virtual_stain_flow software 

In [3]:
sys.path.append(config['paths']['software_path'])
print(str(pathlib.Path('.').absolute().parent.parent))

## Dataset
from virtual_stain_flow.datasets.PatchDataset import PatchDataset
from virtual_stain_flow.datasets.CachedDataset import CachedDataset

## FNet training
from virtual_stain_flow.models.fnet import FNet
from virtual_stain_flow.trainers.Trainer import Trainer

## wGaN training
from virtual_stain_flow.models.unet import UNet
from virtual_stain_flow.models.discriminator import GlobalDiscriminator
from virtual_stain_flow.trainers.WGaNTrainer import WGaNTrainer

## wGaN losses
from virtual_stain_flow.losses.GradientPenaltyLoss import GradientPenaltyLoss
from virtual_stain_flow.losses.DiscriminatorLoss import DiscriminatorLoss
from virtual_stain_flow.losses.GeneratorLoss import GeneratorLoss

from virtual_stain_flow.transforms.MinMaxNormalize import MinMaxNormalize
from virtual_stain_flow.transforms.PixelDepthTransform import PixelDepthTransform

## Metrics
from virtual_stain_flow.metrics.MetricsWrapper import MetricsWrapper
from virtual_stain_flow.metrics.PSNR import PSNR
from virtual_stain_flow.metrics.SSIM import SSIM

## callback
from virtual_stain_flow.callbacks.MlflowLogger import MlflowLogger
from virtual_stain_flow.callbacks.IntermediatePlot import IntermediatePatchPlot

/home/weishanli/Waylab/pediatric_cancer_atlas_analysis


## Define paths and other train parameters

In [4]:
## Loaddata for train
LOADDATA_FILE_PATH = pathlib.Path('.').absolute().parent.parent \
    / '0.data_preprocessing' / 'data_split_loaddata' / 'loaddata_train.csv'
assert LOADDATA_FILE_PATH.exists()
SC_FEATURES_DIR = pathlib.Path(config['paths']['sc_features_path'])

## Output directories
MLFLOW_DIR = pathlib.Path('.').absolute() / 'optuna_mlflow'
MLFLOW_DIR.mkdir(parents=True, exist_ok=True)

OPTUNA_JOBLIB_DIR = pathlib.Path('.').absolute() / 'optuna_joblib'
OPTUNA_JOBLIB_DIR.mkdir(parents=True, exist_ok=True)

## Basic data generation, model convolutional depth and max epoch definition
PATCH_SIZE = 256
DEPTH = 5
EPOCHS = 1_000

## Channels for input and target are read from config
INPUT_CHANNEL_NAMES = config['data']['input_channel_keys']
TARGET_CHANNEL_NAMES = config['data']['target_channel_keys']

## Defines how the train data will be divided to train models on two levels of confluence

In [5]:
DATA_GROUPING = {
    'high_confluence': {
        'seeding_density': [12_000, 8_000]
    },
    'low_confluence': {
        'seeding_density': [4_000, 2_000, 1_000]
    }
}

## Define optimization objectives

In [6]:
import gc
def free_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()

def objective(trial, dataset, channel_name, confluence_group_name):
    
    ## Suggest hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True) # adam optimizer learning rate
    beta0 = trial.suggest_float('beta0', 0.5, 0.9) # adam optimizer beta 0
    beta1 = trial.suggest_float('beta1', 0.9, 0.999) # adam optimizer beta 1
    betas = (beta0, beta1)

    batch_size = trial.suggest_int('batch_size', 16, 64, step=16)
    patience = trial.suggest_int('patience', 5, 20) # early stop patience

    conv_depth = trial.suggest_int('conv_depth', 3, 5) # convolutional depth for fnet model

    ## Setup model and optimizer
    model = FNet(depth=conv_depth)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas)
    
    ## Metrics to be computed (and logged)
    metric_fns = {
        "mse_loss": MetricsWrapper(_metric_name='mse', module=torch.nn.MSELoss()),
        "ssim_loss": SSIM(_metric_name="ssim"),
        "psnr_loss": PSNR(_metric_name="psnr"),
    }

    ## Params to log with mlflow
    params = {
            "lr": lr,
            "beta0": beta0,
            "beta1": beta1,
            "depth": conv_depth,
            "patch_size": PATCH_SIZE,
            "batch_size": batch_size,
            "epochs": EPOCHS,
            "patience": patience,
            "channel_name": channel_name,
        }

    ## mlflow logger callback
    mlflow_logger_callback = MlflowLogger(
        name='mlflow_logger',
        mlflow_uri=MLFLOW_DIR / 'mlruns',
        mlflow_experiment_name=f'FNet_optimize_{confluence_group_name}',
        mlflow_start_run_args={'run_name': f'FNet_optimize_{confluence_group_name}_{channel_name}', 'nested': True},
        mlflow_log_params_args=params
    )
    
    ## Trainer
    trainer = Trainer(
        model = model,
        optimizer = optimizer,
        backprop_loss = torch.nn.L1Loss(), # MAE loss for backpropagation
        dataset = dataset,
        batch_size = batch_size,
        epochs = EPOCHS,
        patience = patience,
        callbacks=[mlflow_logger_callback],
        metrics=metric_fns,
        device = 'cuda',
        early_termination_metric='L1Loss'
    )

    # Train the model and log validation loss
    trainer.train()
    val_loss = trainer.best_loss

    del model
    del optimizer
    del metric_fns
    del mlflow_logger_callback
    del trainer
    
    free_gpu_memory()

    return val_loss

In [ ]:
import joblib
import optuna
import pandas as pd

N_TRIALS = 50  # Total trials
loaddata_df = pd.read_csv(LOADDATA_FILE_PATH)

for confluence_group_name, conditions in DATA_GROUPING.items():

    ## Load dataset
    loaddata_condition_df = loaddata_df.copy()
    for condition, values in conditions.items():
        loaddata_condition_df = loaddata_condition_df[
            loaddata_condition_df[condition].isin(values)
        ]

    sc_features = pd.DataFrame()
    for plate in loaddata_condition_df['Metadata_Plate'].unique():
        sc_features_parquet = SC_FEATURES_DIR / f'{plate}_sc_normalized.parquet'
        if not sc_features_parquet.exists():
            print(f'{sc_features_parquet} does not exist, skipping...')
            continue 
        else:
            sc_features = pd.concat([
                sc_features, 
                pd.read_parquet(
                    sc_features_parquet,
                    columns=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site', 'Metadata_Cells_Location_Center_X', 'Metadata_Cells_Location_Center_Y']
                )
            ])

    ## Create patch dataset
    pds = PatchDataset(
        _loaddata_csv=loaddata_condition_df,
        _sc_feature=sc_features,
        _input_channel_keys=INPUT_CHANNEL_NAMES,
        _target_channel_keys=TARGET_CHANNEL_NAMES,
        _input_transform=PixelDepthTransform(src_bit_depth=16, target_bit_depth=8, _always_apply=True),
        _target_transform=MinMaxNormalize(_normalization_factor=(2 ** 16) - 1, _always_apply=True),
        patch_size=PATCH_SIZE,
        verbose=False,
        patch_generation_method="random_cell",
        n_expected_patches_per_img=50,
        patch_generation_random_seed=42
    )

    for channel_name in TARGET_CHANNEL_NAMES:

        ## Cache dataset of channel
        pds.set_input_channel_keys(INPUT_CHANNEL_NAMES)
        pds.set_target_channel_keys(channel_name)
        cds = CachedDataset(
            dataset=pds,
            prefill_cache=True
        )

        print(f"Beginning optimization for channel: {channel_name} for {confluence_group_name}")

        # Load the existing study for the current channel
        study_path = OPTUNA_JOBLIB_DIR / f"FNet_optimize_{channel_name}_{confluence_group_name}.joblib"

        if study_path.exists():
            study = joblib.load(study_path)
            completed_trials = len(study.trials)  # completed trials
        else:
            study = optuna.create_study(
                direction="minimize",
                study_name=f"FNet_optimize_{channel_name}_{confluence_group_name}",
                sampler=optuna.samplers.TPESampler(seed=42)
            )
            completed_trials = 0  # New experiment
            
        # Resume optimization from where it left off
        trials_remaining = N_TRIALS - completed_trials
        if trials_remaining > 0:
            for _ in range(trials_remaining):
                study.optimize(lambda trial: objective(trial, cds, channel_name, confluence_group_name), n_trials=1)
                joblib.dump(study, study_path)  # Save study after every trial
                print(f"Saved study after trial {len(study.trials)}/{N_TRIALS}")

        # Print best trial results
        print(f"Best trial for channel {channel_name}:")
        print(f"  Validation Loss: {study.best_trial.value}")
        print(f"  Hyperparameters: {study.best_trial.params}")

Beginning optimization for channel: OrigDNA for high_confluence
Best trial for channel OrigDNA:
  Validation Loss: 0.016428549215197562
  Hyperparameters: {'lr': 0.0027501521311678614, 'beta0': 0.8688225992072988, 'beta1': 0.9654164885801781, 'batch_size': 48, 'patience': 16, 'conv_depth': 4}
Beginning optimization for channel: OrigER for high_confluence
Best trial for channel OrigER:
  Validation Loss: 0.020866319803254946
  Hyperparameters: {'lr': 0.00044118732609755304, 'beta0': 0.8125413329356658, 'beta1': 0.9146557494825732, 'batch_size': 32, 'patience': 12, 'conv_depth': 3}
Beginning optimization for channel: OrigAGP for high_confluence
Best trial for channel OrigAGP:
  Validation Loss: 0.0029262731383953777
  Hyperparameters: {'lr': 0.0005690482786963295, 'beta0': 0.8951555358866697, 'beta1': 0.9770467004930141, 'batch_size': 32, 'patience': 15, 'conv_depth': 4}
Beginning optimization for channel: OrigMito for high_confluence
Best trial for channel OrigMito:
  Validation Loss: 0

[I 2025-02-24 09:50:30,158] Trial 30 finished with value: 0.019931084609457424 and parameters: {'lr': 0.0003023080480977387, 'beta0': 0.6641587946853241, 'beta1': 0.9619633462077796, 'batch_size': 32, 'patience': 8, 'conv_depth': 4}. Best is trial 22 with value: 0.018208095517296057.


Saved study after trial 31/50
Early termination at epoch 21 with best validation metric 0.018611485425096292


[I 2025-02-24 09:52:54,020] Trial 31 finished with value: 0.018611485425096292 and parameters: {'lr': 0.0002887756125661432, 'beta0': 0.6079006601570802, 'beta1': 0.9520948147300842, 'batch_size': 16, 'patience': 6, 'conv_depth': 4}. Best is trial 22 with value: 0.018208095517296057.


Saved study after trial 32/50
Early termination at epoch 16 with best validation metric 0.019879762942974385


...

Saved study after trial 35/50
Early termination at epoch 17 with best validation metric 0.01902948721097066


[I 2025-02-24 10:02:54,162] Trial 35 finished with value: 0.01902948721097066 and parameters: {'lr': 0.0002388932800400586, 'beta0': 0.5841576203574859, 'beta1': 0.9693987250418548, 'batch_size': 16, 'patience': 6, 'conv_depth': 4}. Best is trial 22 with value: 0.018208095517296057.


Saved study after trial 36/50
Early termination at epoch 24 with best validation metric 0.019311377492088538


[I 2025-02-24 10:05:54,404] Trial 36 finished with value: 0.019311377492088538 and parameters: {'lr': 0.00014902646085335333, 'beta0': 0.5622147649221322, 'beta1': 0.9782604091267229, 'batch_size': 16, 'patience': 8, 'conv_depth': 5}. Best is trial 22 with value: 0.018208095517296057.


Saved study after trial 37/50


In [ ]:
# RUN_EXTRA = False
# loaddata_df = pd.read_csv(LOADDATA_FILE_PATH)

# for confluence_group_name, conditions in DATA_GROUPING.items():

#     ## Load dataset
#     loaddata_condition_df = loaddata_df.copy()
#     for condition, values in conditions.items():
#         loaddata_condition_df = loaddata_condition_df[
#             loaddata_condition_df[condition].isin(values)
#         ]

#     sc_features = pd.DataFrame()
#     for plate in loaddata_condition_df['Metadata_Plate'].unique():
#         sc_features_parquet = SC_FEATURES_DIR / f'{plate}_sc_normalized.parquet'
#         if not sc_features_parquet.exists():
#             print(f'{sc_features_parquet} does not exist, skipping...')
#             continue 
#         else:
#             sc_features = pd.concat([
#                 sc_features, 
#                 pd.read_parquet(
#                     sc_features_parquet,
#                     columns=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site', 'Metadata_Cells_Location_Center_X', 'Metadata_Cells_Location_Center_Y']
#                 )
#             ])

#     ## Create patch dataset
#     pds = PatchDataset(
#         _loaddata_csv=loaddata_condition_df,
#         _sc_feature=sc_features,
#         _input_channel_keys=INPUT_CHANNEL_NAMES,
#         _target_channel_keys=TARGET_CHANNEL_NAMES,
#         _input_transform=PixelDepthTransform(src_bit_depth=16, target_bit_depth=8, _always_apply=True),
#         _target_transform=MinMaxNormalize(_normalization_factor=(2 ** 16) - 1, _always_apply=True),
#         patch_size=PATCH_SIZE,
#         verbose=False,
#         patch_generation_method="random_cell",
#         n_expected_patches_per_img=50,
#         patch_generation_random_seed=42
#     )

#     for channel_name in TARGET_CHANNEL_NAMES:

#         ## Cache dataset of channel
#         pds.set_input_channel_keys(INPUT_CHANNEL_NAMES)
#         pds.set_target_channel_keys(channel_name)
#         cds = CachedDataset(
#             dataset=pds,
#             prefill_cache=True
#         )

#         print(f"Beginning optimization for channel: {channel_name} for {confluence_group_name}")

#         # Load the existing study for the current channel
#         study_path = OPTUNA_JOBLIB_DIR / f"FNet_optimize_{channel_name}_{confluence_group_name}.joblib"
#         if study_path.exists():
#             if RUN_EXTRA:
#                 study = joblib.load(study_path)
#             else:
#                 print("Skipping optimization due to existing joblib...")
#                 continue
#         else:
#             study = optuna.create_study(
#                 direction="minimize",
#                 study_name=f"FNet_optimize_{channel_name}_{confluence_group_name}",
#                 sampler=optuna.samplers.TPESampler(seed=42)
#             )

#         # Optimize the objective function for the current channel
#         study.optimize(lambda trial: objective(trial, cds, channel_name, confluence_group_name), n_trials=50)

#         # Save the updated study for the current channel
#         joblib.dump(study, study_path)

#         # Print best trial results
#         print(f"Best trial for channel {channel_name}:")
#         print(f"  Validation Loss: {study.best_trial.value}")
#         print(f"  Hyperparameters: {study.best_trial.params}")